<a href="https://colab.research.google.com/github/dfridland/PyTorch/blob/HW3/PyTorchHW03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

import torch
import torch.nn as nn
from sklearn.metrics import r2_score
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy import stats
import warnings
import math
warnings.filterwarnings('ignore')

In [73]:
from sklearn.datasets import fetch_california_housing


data = fetch_california_housing()

In [74]:
class MyDataset(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, init_target, transform=None):
        self._base_dataset = torch.from_numpy(init_dataset).type(torch.float)
        self._base_targets = torch.from_numpy(init_target).type(torch.float)
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        features = self._base_dataset[idx]
        target = self._base_targets[idx]

        if self.transform is not None:
            features = self.transform(features)
      
        return features, target



In [75]:
class Perceptron(nn.Module):
  def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation

  def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "leaky_relu":
            return F.leaky_relu(x)
        if self.activation == "sigmoid":
            return F.sigmoid(x)
        raise RuntimeError

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim, 'leaky_relu')
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1, "sigmoid")
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)

        return x
     

In [76]:
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size = 0.25, random_state = 13)


In [77]:
train_dataset = MyDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=150, shuffle=False)

In [78]:
test_dataset = MyDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

In [79]:
# class FeedForward(nn.Module):
#     def __init__(self, input_dim, hidden_dim):
#         super(FeedForward, self).__init__()
#         self.bn1 = nn.BatchNorm1d(input_dim)
#         self.fc1 = Perceptron(input_dim, hidden_dim)
#         self.bn2 = nn.BatchNorm1d(hidden_dim)
#         self.dp = nn.Dropout(0.25)
#         self.fc2 = Perceptron(hidden_dim, 1, "sigmoid")
        
#     def forward(self, x):
#         x = self.bn1(x)
#         x = self.fc1(x)
#         x = self.dp(x)
#         x = self.fc2(x)
#         return x.view(-1)


In [80]:
net = FeedForward(8, 50)
net


FeedForward(
  (fc1): Perceptron(
    (fc): Linear(in_features=8, out_features=50, bias=True)
  )
  (bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.25, inplace=False)
  (fc2): Perceptron(
    (fc): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [84]:
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
# optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001)

criterion = nn.MSELoss()

In [82]:
if torch.cuda.is_available():
    net.cuda()

In [83]:

num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, r2 = 0.0, 0.0, 0.0

    for i, data in enumerate(train_loader):
        fets, target = data[0], data[1]

        optimizer.zero_grad()

        outputs = net(fets)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_items += len(target)

        predict = outputs.data.numpy()
        train_target = target.view(target.shape[0], 1).numpy()
        r2 += r2_score(train_target, predict)
        
        if i % 30 == 0:
            net.eval()

            data = list(test_loader)[0]

            test_outputs = net(data[0])
            test_predict = test_outputs.data.numpy()
            test_target = data[1].view(data[1].shape[0], 1)
            test_r2 = r2_score(test_target, test_predict)

            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'r2: {r2:.3f}. ' \
                  f'Test r2: {test_r2:.3f}')

            running_loss, running_items, r2 = 0.0, 0.0, 0.0

            net.train()
        
print("Training  is done!")
     

Epoch [1/10]. Step [1/104]. Loss: 0.028. r2: -1.794. Test r2: -2.030
Epoch [1/10]. Step [31/104]. Loss: 0.024. r2: -54.051. Test r2: -2.475
Epoch [1/10]. Step [61/104]. Loss: 0.024. r2: -49.329. Test r2: -2.151
Epoch [1/10]. Step [91/104]. Loss: 0.022. r2: -45.697. Test r2: -1.781
Epoch [2/10]. Step [1/104]. Loss: 0.024. r2: -1.407. Test r2: -1.610
Epoch [2/10]. Step [31/104]. Loss: 0.020. r2: -39.585. Test r2: -1.342
Epoch [2/10]. Step [61/104]. Loss: 0.019. r2: -34.366. Test r2: -1.148
Epoch [2/10]. Step [91/104]. Loss: 0.018. r2: -31.648. Test r2: -1.057
Epoch [3/10]. Step [1/104]. Loss: 0.020. r2: -1.006. Test r2: -1.025
Epoch [3/10]. Step [31/104]. Loss: 0.017. r2: -29.773. Test r2: -0.979
Epoch [3/10]. Step [61/104]. Loss: 0.017. r2: -27.901. Test r2: -0.955
Epoch [3/10]. Step [91/104]. Loss: 0.017. r2: -27.477. Test r2: -0.944
Epoch [4/10]. Step [1/104]. Loss: 0.019. r2: -0.931. Test r2: -0.933
Epoch [4/10]. Step [31/104]. Loss: 0.016. r2: -27.647. Test r2: -0.926
Epoch [4/10]. 

In [85]:
# Adam and RMS are optimizators of choice. 